#### Available scene list

In [1]:
import json


f = open('../../01/01_scene_list/scene.json', "r")
scene_json = json.load(f)

# Available scene list
scene_list = list(scene_json.keys())

# Coordinate conversion
scene_dict = scene_json

f.close()

#### Process CSV

In [2]:
import os
import pandas as pd


def process_scene_df(scene_id):
    
    ds_pth = "E:\datasets\iGibson-dataset\position_rd_no_obj/"
    ds_pth = f"{ds_pth}/{scene_id}/"
    sp_nums = len(os.listdir(ds_pth))
    
    mp_pth = "E:\datasets\iGibson-dataset\scene_map_processed/"
    mp_pth = f"{mp_pth}/{scene_id}/{scene_id}.png"
    
    tgt_pth = f"E:\datasets\iGibson-dataset\position_csv/"
    tgt_pth = f"{tgt_pth}/{scene_id}.csv"
    
    scene_df = {
        "gt": [],
        "rgb": [],
        "d": [],
        "mp": []
    }
    
    for idx in range(sp_nums):
        
        gt_pth = f"{ds_pth}/{idx}/gt/"
        scene_df["gt"].append(gt_pth)
        
        rgb_pth = f"{ds_pth}/{idx}/rgb/"
        scene_df["rgb"].append(rgb_pth)
        
        d_pth = f"{ds_pth}/{idx}/d/"
        scene_df["d"].append(d_pth)
        
        scene_df["mp"].append(mp_pth)
        
    scene_df = pd.DataFrame(scene_df)
    scene_df.to_csv(tgt_pth, index=False)
    
    
def train_test_split(cfg):
    
    tgt_pth = f"E:\datasets\iGibson-dataset\position_csv/"
    
    # Merge train scene csv
    train_df = pd.DataFrame()
    train_list = cfg["train"]
    
    for train_idx in train_list:
        
        scene_id = scene_list[train_idx]
        scene_csv = f"{tgt_pth}/{scene_id}.csv"
        scene_df = pd.read_csv(scene_csv)
        
        train_df = pd.concat((train_df, scene_df))
        
    train_df.to_csv(f"{tgt_pth}/train.csv", index=False)
        
    # Merge test scene csv
    test_df = pd.DataFrame()
    test_list = cfg["test"]
    
    for test_idx in test_list:
        
        scene_id = scene_list[test_idx]
        scene_csv = f"{tgt_pth}/{scene_id}.csv"
        scene_df = pd.read_csv(scene_csv)
        
        test_df = pd.concat((test_df, scene_df))
        
    test_df.to_csv(f"{tgt_pth}/test.csv", index=False)
    

In [3]:
# Generate directory csv for each scene
s_idx_list = [1, 3, 4, 5, 6]

for s_idx in s_idx_list:
    
    print(scene_list[s_idx])
    process_scene_df(scene_list[s_idx])
    

# Split train and test dataset    
cfg = {
    "train": [
        1, 3, 4, 5
    ],
    "test": [
        6
    ]
}

train_test_split(cfg)


Beechwood_1_int
Ihlen_0_int
Ihlen_1_int
Merom_0_int
Merom_1_int


#### Image map and image observation

In [4]:
from torch import nn

import pandas as pd

from utils import read_gt_position
from utils import read_d_position, read_rgb_position
from utils import read_mp_image


class PositionIMIO(nn.Module):
    
    def __init__(self, data_pth):
        
        super(PositionIMIO, self).__init__()
        
        self.data = pd.read_csv(data_pth)
        
    def __len__(self):
        return len(self.data)
        
    def __getitem__(self, index):
        
        # Read ground truth
        gt_pth = self.data["gt"][index]
        gt = read_gt_position(gt_pth)
        gt = gt[0, ...]
        
        # Read observation
        # obs_pth = self.data["rgb"][index]
        # obs = read_rgb_position(obs_pth)
        
        obs_pth = self.data["d"][index]
        obs = read_d_position(obs_pth)
        
        # Read map
        mp_pth = self.data["mp"][index]
        mp = read_mp_image(mp_pth)
        
        
        return (gt, obs, mp)
        

In [5]:
from torch.utils.data import DataLoader


train_pth = "e:\datasets\iGibson-dataset\position_csv/train.csv"
train_dataset = PositionIMIO(train_pth)
print(f"Train dataset has {len(train_dataset)} samples")

train_dataloader = DataLoader(train_dataset, batch_size=4, shuffle=True)

gt, obs, mp = next(iter(train_dataloader))
print(gt.shape)
print(obs.shape)
print(mp.shape)

for data in train_dataloader:
    pass

test_pth = "e:\datasets\iGibson-dataset\position_csv/test.csv"
test_dataset = PositionIMIO(test_pth)
print(f"Test dataset has {len(test_dataset)} samples")

test_dataloader = DataLoader(test_dataset, batch_size=4, shuffle=True)

gt, obs, mp = next(iter(test_dataloader))
print(gt.shape)
print(obs.shape)
print(mp.shape)
for data in train_dataloader:
    pass

Train dataset has 80 samples
torch.Size([4])
torch.Size([4, 8, 3, 224, 224])
torch.Size([4, 3, 224, 224])
Test dataset has 20 samples
torch.Size([4])
torch.Size([4, 8, 3, 224, 224])
torch.Size([4, 3, 224, 224])
